In [1]:
%%bash
# Go to folder containing the source code
cd /app/mialsuperresolutiontoolkit/
# Install the pymialsrtk package inside the python/conda environment
python setup.py install --force

running install
running build
running build_py
copying pymialsrtk/interfaces/preprocess.py -> build/lib/pymialsrtk/interfaces
running build_scripts
running install_lib
copying build/lib/pymialsrtk/info.py -> /opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/pymialsrtk
copying build/lib/pymialsrtk/interfaces/registration.py -> /opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/pymialsrtk/interfaces
copying build/lib/pymialsrtk/interfaces/reconstruction.py -> /opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/pymialsrtk/interfaces
copying build/lib/pymialsrtk/interfaces/preprocess.py -> /opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/pymialsrtk/interfaces
copying build/lib/pymialsrtk/interfaces/__init__.py -> /opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/pymialsrtk/interfaces
copying build/lib/pymialsrtk/interfaces/utils.py -> /opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/pymialsrtk/interfaces
copying build/lib/pymialsrtk/__in

In [2]:
import os
import json
import shutil

# Imports from nipype
from nipype.interfaces.io import BIDSDataGrabber
from nipype.pipeline import Node, Workflow

# Import the implemented interface from pymialsrtk
import pymialsrtk.interfaces.preprocess as preprocess


In [3]:
# Set different variables (defined in cell 2) such that we do not have to rerun cell 2
bids_dir = os.path.join('/fetaldata')
# output_dir = os.path.join('/fetaldata','derivatives')
# mask_dir = os.path.join('/fetaldata','derivatives','manual_masks')
subject = '01'

# #Make sure we start from empty derivatives
# if os.path.exists(output_dir):
#     shutil.rmtree(output_dir)
    

# #Check if mandatory derivatives dataset_description.json exists in derivatives/mialsrtk.
# # If not, it is created before running the workflow, otherwise BIDSDataGrabber is not happy and raises an error. 

# mialsrtk_dataset_description_json = os.path.join(output_dir,'dataset_description.json')

# print('Check for {}'.format(mialsrtk_dataset_description_json))
# if not os.access(mialsrtk_dataset_description_json, os.R_OK):
#     data = {'PipelineDescription':{'Name': 'MIAL Super-Resolution ToolKit', 
#                                 'Version': 'v2.0.0-beta', 
#                                 'CodeURL': 'https://github.com/sebastientourbier/mialsuperresolutiontoolkit'
#                                   },
#             'Name': 'MIAL Super-Resolution ToolKit',
#             'BIDSVersion': '1.2.0'
#            }
#     os.makedirs(output_dir)
#     with open(mialsrtk_dataset_description_json, 'w+') as outfile:
#         json.dump(data, outfile, indent=4)
#     print('File {} was created'.format(mialsrtk_dataset_description_json))
# else:
#     print('File {} already exists'.format(mialsrtk_dataset_description_json))

In [4]:
from nipype import config, logging

In [5]:
## Node linkage
def create_workflow(bids_dir,subject):
#     wf_base_dir = os.path.join("{}".format(output_dir),"superres-mri","sub-{}".format(subject),"nipype")
    output_dir = os.path.join("{}".format(bids_dir),"derivatives","superres-mri")
    wf_base_dir = os.path.join(output_dir,"sub-{}".format(subject))

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    print("Ouput directory: {}".format(wf_base_dir))

    wf = Workflow(name="srr_preproc",base_dir=wf_base_dir)
    
    srr_preproc_dir = os.path.join(wf.base_dir, wf.name )
    
    
    # Initialization
    if os.path.isfile(os.path.join(output_dir,"pypeline.log")):
        os.unlink(os.path.join(output_dir,"pypeline.log"))
#         open(os.path.join(output_dir,"pypeline.log"), 'a').close()
        

    config.update_config({'logging': {'log_directory': os.path.join(output_dir),
                              'log_to_file': True},
                          'execution': {
                            'remove_unnecessary_outputs': False,
                            'stop_on_first_crash': True,
                            'stop_on_first_rerun': False,
                            'crashfile_format': "txt",
                            'write_provenance' : False,
                            },
                          'monitoring': {
                            'enabled': True
                            }
                        })
    
    logging.update_logging(config)
    iflogger = logging.getLogger('nipype.interface')

    iflogger.info("**** Processing ****")

    bg = Node(BIDSDataGrabber(infields = ['subject']),name='bids_grabber')
    bg.inputs.base_dir = bids_dir
    bg.inputs.subject = subject
    bg.inputs.index_derivatives = True
    bg.inputs.output_query = {'T2ws': dict(suffix='T2w',datatype='anat',extensions=[".nii",".nii.gz"]),
                              'masks': dict(suffix='mask',datatype='anat',extensions=[".nii",".nii.gz"])}

        
    nlmDenoise = Node(interface=preprocess.MultipleBtkNLMDenoising(),name='nlmDenoise')
    nlmDenoise.inputs.bids_dir = bids_dir
    nlmDenoise.inputs.weight = 1
    
    
    srtkCorrectSliceIntensity01_nlm = Node(interface=preprocess.MultipleMialsrtkCorrectSliceIntensity(),name='srtkCorrectSliceIntensity01_nlm')
    srtkCorrectSliceIntensity01_nlm.inputs.bids_dir = bids_dir

    srtkCorrectSliceIntensity01 = Node(interface=preprocess.MultipleMialsrtkCorrectSliceIntensity(),name='srtkCorrectSliceIntensity01')
    srtkCorrectSliceIntensity01.inputs.bids_dir = bids_dir

    
    srtkSliceBySliceN4BiasFieldCorrection = Node(interface=preprocess.MultipleMialsrtkSliceBySliceN4BiasFieldCorrection(), name='srtkSliceBySliceN4BiasFieldCorrection')
    srtkSliceBySliceN4BiasFieldCorrection.inputs.bids_dir = bids_dir
    
    srtkSliceBySliceCorrectBiasField = Node(interface=preprocess.MultipleMialsrtkSliceBySliceCorrectBiasField(), name='srtkSliceBySliceCorrectBiasField')
    srtkSliceBySliceCorrectBiasField.inputs.bids_dir = bids_dir
    
    
    srtkCorrectSliceIntensity02_nlm = Node(interface=preprocess.MultipleMialsrtkCorrectSliceIntensity(),name='srtkCorrectSliceIntensity02_nlm')
    srtkCorrectSliceIntensity02_nlm.inputs.bids_dir = bids_dir

    srtkCorrectSliceIntensity02 = Node(interface=preprocess.MultipleMialsrtkCorrectSliceIntensity(),name='srtkCorrectSliceIntensity02')
    srtkCorrectSliceIntensity02.inputs.bids_dir = bids_dir
    
    
    srtkIntensityStandardization = Node(interface=preprocess.MialsrtkIntensityStandardization(),name='srtkIntensityStandardization')
    srtkIntensityStandardization.inputs.bids_dir = bids_dir
    
    srtkIntensityStandardization_nlm = Node(interface=preprocess.MialsrtkIntensityStandardization(),name='srtkIntensityStandardization_nlm')
    srtkIntensityStandardization_nlm.inputs.bids_dir = bids_dir
    
    
    wf.connect(bg, "T2ws", nlmDenoise, "input_images")
    wf.connect(bg, "masks", nlmDenoise, "input_masks")
    
    wf.connect(nlmDenoise, "output_images", srtkCorrectSliceIntensity01_nlm, "input_images")
    wf.connect(bg, "masks", srtkCorrectSliceIntensity01_nlm, "input_masks")
    
    wf.connect(bg, "T2ws", srtkCorrectSliceIntensity01, "input_images")
    wf.connect(bg, "masks", srtkCorrectSliceIntensity01, "input_masks")
    
    wf.connect(srtkCorrectSliceIntensity01_nlm, "output_images", srtkSliceBySliceN4BiasFieldCorrection, "input_images")
    wf.connect(bg, "masks", srtkSliceBySliceN4BiasFieldCorrection, "input_masks")
    
    wf.connect(srtkCorrectSliceIntensity01, "output_images", srtkSliceBySliceCorrectBiasField, "input_images")
    wf.connect(srtkSliceBySliceN4BiasFieldCorrection, "output_fields", srtkSliceBySliceCorrectBiasField, "input_fields")
    wf.connect(bg, "masks", srtkSliceBySliceCorrectBiasField, "input_masks")
    
    
    wf.connect(srtkSliceBySliceCorrectBiasField, "output_images", srtkCorrectSliceIntensity02, "input_images")
    wf.connect(bg, "masks", srtkCorrectSliceIntensity02, "input_masks")
    
    wf.connect(srtkSliceBySliceN4BiasFieldCorrection, "output_images", srtkCorrectSliceIntensity02_nlm, "input_images")
    wf.connect(bg, "masks", srtkCorrectSliceIntensity02_nlm, "input_masks")
    
    
    
    wf.connect(srtkCorrectSliceIntensity02, "output_images", srtkIntensityStandardization, "input_images")
        
    wf.connect(srtkCorrectSliceIntensity02_nlm, "output_images", srtkIntensityStandardization_nlm, "input_images")
    
    
    

    return wf

In [6]:
m_wf = create_workflow(bids_dir, subject)
m_wf.run()
m_wf.write_graph()

Ouput directory: /fetaldata/derivatives/superres-mri/sub-01
200114-18:06:15,616 nipype.interface INFO:
	 **** Processing ****
200114-18:06:15,637 nipype.workflow INFO:
	 Workflow srr_preproc settings: ['check', 'execution', 'logging', 'monitoring']
200114-18:06:15,644 nipype.workflow INFO:
	 Running serially.
200114-18:06:15,645 nipype.workflow INFO:
	 [Node] Setting-up "srr_preproc.bids_grabber" in "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/bids_grabber".
200114-18:06:15,648 nipype.workflow INFO:
	 [Node] Running "bids_grabber" ("nipype.interfaces.io.BIDSDataGrabber")


/opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/bids/layout/layout.py:665: UserWarning: In pybids 0.9.0, the 'extensions' filter was deprecated in favor of 'extension'. The former will stop working in 0.11.0.
  warnings.warn("In pybids 0.9.0, the 'extensions' filter was "


200114-18:06:17,87 nipype.workflow INFO:
	 [Node] Finished "srr_preproc.bids_grabber".
200114-18:06:17,88 nipype.workflow INFO:
	 [Node] Setting-up "srr_preproc.srtkCorrectSliceIntensity01" in "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkCorrectSliceIntensity01".
200114-18:06:17,90 nipype.workflow INFO:
	 [Node] Cached "srr_preproc.srtkCorrectSliceIntensity01" - collecting precomputed outputs
200114-18:06:17,91 nipype.workflow INFO:
	 [Node] "srr_preproc.srtkCorrectSliceIntensity01" found cached.
200114-18:06:17,91 nipype.workflow INFO:
	 [Node] Setting-up "srr_preproc.nlmDenoise" in "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/nlmDenoise".
200114-18:06:17,94 nipype.workflow INFO:
	 [Node] Cached "srr_preproc.nlmDenoise" - collecting precomputed outputs
200114-18:06:17,95 nipype.workflow INFO:
	 [Node] "srr_preproc.nlmDenoise" found cached.
200114-18:06:17,95 nipype.workflow INFO:
	 [Node] Setting-up "srr_preproc.srtkCorrectSliceIntensity01_nlm" in "/fetaldata

'/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/graph.png'

In [7]:
m_wf.write_graph()

200114-18:06:17,407 nipype.workflow INFO:
	 Generated workflow graph: /fetaldata/derivatives/superres-mri/sub-01/srr_preproc/graph.png (graph2use=hierarchical, simple_form=True).


'/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/graph.png'